In [1]:
import riak

In [2]:
# connect to database
myClient = riak.RiakClient()
myClient.ping()

True

In [3]:
!echo learner | sudo -S -p "" riak-admin bucket-type create sets '{"props":{"datatype":"set"}}'

sets created

can no longer be downgraded to a version of Riak prior to 2.0


In [4]:
!echo learner | sudo -S -p "" riak-admin bucket-type activate sets

sets has been activated

downgraded to a version of Riak prior to 2.0


In [5]:
BUCKET_TWEETS = 'tweets'
BUCKET_USERS = 'users'
BUCKET_HASHTAGS = 'hashtags'

tweets = myClient.bucket(BUCKET_TWEETS)
users = myClient.bucket(BUCKET_USERS)
hashtags = myClient.bucket_type('sets').bucket(BUCKET_HASHTAGS)


In [6]:
def insert_hashtag(hashtag, tweet_id):  
    hashtags_set = hashtags.get(hashtag.encode('utf8'))
    hashtags_set.add(tweet_id.encode('utf8'))    
    hashtags_set.store()


In [8]:
def insert_user(user_json, replace = True):
    if replace or not users.get(user_json["id_str"]).exists:
        user = users.new(user_json["id_str"], user_json)
        user.store()
    else:
        user = users.get(user_json["id_str"])
    return user

In [9]:
def insert_tweet(tweet_json):    
    tweet = tweets.new(tweet_json["id_str"], tweet_json)
    user = insert_user(tweet_json['user'])
    
    tweet.add_link(user, tag="TWITTED_BY")
    tweet.store()
        
    for user_mentioned in tweet_json['user_mentions']:
        insert_user(user_mentioned, replace = False)         

    if 'retweeted_status' in tweet_json:
        insert_tweet(tweet_json['retweeted_status'])
        
    for hashtag in tweet_json['entities']:
        insert_hashtag(hashtag, tweet_json["id_str"])
        
#pp (tweet_json)

In [10]:
import json
from pprintpp import pprint as pp

tweets_data_path = '../data/tweets.json'

tweets_file = open(tweets_data_path, "r")

i = 1
for line in tweets_file:
    i = i + 1
    if i == 4:
        insert_tweet(json.loads(line))
        break

In [11]:
!curl -s http://localhost:8098/riak/users/2318606822 

{"favourites_count": 1425, "description": "We cover #BigData, #FinTech and #IoT - Looking at how data science and connected devices are changing technology.", "friends_count": 2505, "created_at": "Thu Jan 30 09:46:50 +0000 2014", "time_zone": "Amsterdam", "profile_image_url": "http://pbs.twimg.com/profile_images/634739150382436353/JIjlGeGO_normal.png", "name": "Dataconomy", "followers_count": 8801, "id_str": "2318606822", "statuses_count": 11179, "geo_enabled": true, "id": 2318606822, "screen_name": "DataconomyMedia"}

In [12]:
print users.get("2318606822").data

{u'favourites_count': 1425, u'description': u'We cover #BigData, #FinTech and #IoT - Looking at how data science and connected devices are changing technology.', u'friends_count': 2505, u'created_at': u'Thu Jan 30 09:46:50 +0000 2014', u'time_zone': u'Amsterdam', u'profile_image_url': u'http://pbs.twimg.com/profile_images/634739150382436353/JIjlGeGO_normal.png', u'followers_count': 8801, u'screen_name': u'DataconomyMedia', u'id_str': u'2318606822', u'statuses_count': 11179, u'geo_enabled': True, u'id': 2318606822, u'name': u'Dataconomy'}


In [13]:
!curl  http://localhost:8098/types/sets/buckets/hashtags/datatypes/NoSQL 

{"type":"set","value":["658168624692228096","658171535824416768"],"context":"g2wAAAABaAJtAAAADCMJ/vkYKKwZAAAnEWECag=="}

In [14]:
!curl  http://localhost:8098/types/sets/buckets/hashtags/datatypes/SQL 

{"type":"set","value":["658168624692228096","658171535824416768"],"context":"g2wAAAABaAJtAAAADCMJ/vkYEMqbAAAnEWECag=="}

In [15]:
print hashtags.get("SQL")

frozenset(['658168624692228096', '658171535824416768'])


In [16]:
!curl -s http://localhost:8098/riak/tweets/658171535824416768 

{"lang": "en", "retweeted_status": {"lang": "en", "entities": ["SQL", "NoSQL"], "text": "Coming Full Circle: Why #SQL now powers the #NoSQL Craze, by Ryan Betts of @VoltDB https://t.co/sj11zLPgEX", "created_at": "Sun Oct 25 06:30:02 +0000 2015", "user_mentions": [{"screen_name": "VoltDB", "id": 97716631, "name": "VoltDB", "id_str": "97716631"}], "source": "<a href=\"http://bufferapp.com\" rel=\"nofollow\">Buffer</a>", "id_str": "658168624692228096", "retweet_count": 1, "id": 658168624692228096, "favorite_count": 0, "user": {"favourites_count": 1425, "description": "We cover #BigData, #FinTech and #IoT - Looking at how data science and connected devices are changing technology.", "friends_count": 2505, "created_at": "Thu Jan 30 09:46:50 +0000 2014", "time_zone": "Amsterdam", "profile_image_url": "http://pbs.twimg.com/profile_images/634739150382436353/JIjlGeGO_normal.png", "name": "Dataconomy", "followers_count": 8801, "id_str": "2318606822", "statuses_count": 11179, "geo_enabled": true,

In [17]:
print tweets.get("658171535824416768").data

{u'lang': u'en', u'retweeted_status': {u'lang': u'en', u'source': u'<a href="http://bufferapp.com" rel="nofollow">Buffer</a>', u'text': u'Coming Full Circle: Why #SQL now powers the #NoSQL Craze, by Ryan Betts of @VoltDB https://t.co/sj11zLPgEX', u'created_at': u'Sun Oct 25 06:30:02 +0000 2015', u'user_mentions': [{u'id_str': u'97716631', u'screen_name': u'VoltDB', u'name': u'VoltDB', u'id': 97716631}], u'entities': [u'SQL', u'NoSQL'], u'id_str': u'658168624692228096', u'retweet_count': 1, u'id': 658168624692228096, u'favorite_count': 0, u'user': {u'favourites_count': 1425, u'description': u'We cover #BigData, #FinTech and #IoT - Looking at how data science and connected devices are changing technology.', u'friends_count': 2505, u'created_at': u'Thu Jan 30 09:46:50 +0000 2014', u'time_zone': u'Amsterdam', u'profile_image_url': u'http://pbs.twimg.com/profile_images/634739150382436353/JIjlGeGO_normal.png', u'followers_count': 8801, u'screen_name': u'DataconomyMedia', u'id_str': u'2318606

In [27]:
for tweet_id in hashtags.get("SQL"):
    tweet = tweets.get(tweet_id)
    user_link = tweet.links[0]
    if user_link <> None:
        tag = user_link[2]
        bucket = user_link[0]
        key = user_link[1]
        if tag == "TWITTED_BY":
            user = myClient.bucket(bucket).get(key)
        
    
    print user.data["screen_name"] + "-> " + tweet.data["text"]

DataconomyMedia-> Coming Full Circle: Why #SQL now powers the #NoSQL Craze, by Ryan Betts of @VoltDB https://t.co/sj11zLPgEX
MAGISTR_OM-> RT @DataconomyMedia: Coming Full Circle: Why #SQL now powers the #NoSQL Craze, by Ryan Betts of @VoltDB https://t.co/sj11zLPgEX
